In [1]:
import os
from litellm import completion

In [3]:
open_ai_key = os.environ["OPENAI_API_KEY"]

In [4]:
os.environ["AWS_REGION_NAME"] = "us-east-1"


In [5]:
test_q = [{'role': 'system',
   'content': "You are an expert in Salesforce and you have access to a Salesforce instance. \n\n# Instructions\n- You will be provided a question, the system description, and relevant task context.\n- Interact with the Salesforce instance to build Salesforce Object Query Language (SOQL) or Salesforce Object Search Language (SOSL) queries as appropriate, to help you answer the question.\n- Salesforce Object Search Language (SOSL) can be used to construct text-based search queries against the search index.\n- Your generation should always be an Action command and NOTHING ELSE. Generate only one Action command.\n- DO NOT generate ANY system observation, you will receive this based on your Action command.\n- If no record is found matching the requirements mentioned, just return 'None'.\n- If the user's request is unclear or under-specified, use the respond action to ask for clarification before proceeding with queries or submitting an answer.\n\nHere is a description of how to use this command:\n## Action\n- Can be 'execute' or 'submit'.\n- execute, to execute SOQL that will return the observation from running the query on the Salesforce instance.\n- submit, to return the final answer of the task to the user.\n- Format: <execute> a valid SOQL query </execute> or <respond> user response </respond>\n\n# Guidelines\n- Execute SOQL/SOSL queries to understand the Salesforce instance that will help you find the answer to the question.\n- When you are confident about the answer, submit it.\n- Always end with a submit action containing ONLY the answer, NO full sentence or any explanation.\n- If no record is found matching the requirements mentioned, just return 'None'.\n\n# Example 1\nQuestion: What is the total number of opportunities?\nOutput:\n<execute> SELECT COUNT() FROM Opportunity </execute>\n     (If the observation from the Salesforce instance 100, your next step can be)\n<respond> 100 </respond> OR <respond> The total number of opportunities is 100 </respond>\n\n# Example 2\nQuestion: Look for the name Joe Smith in the name field of a lead and return the name and phone number.\nOutput:\n<execute> FIND {Joe Smith} IN NAME FIELDS RETURNING Lead(Name, Phone) </execute>\n    (If the observation from the Salesforce instance is [{Joe Smith, 1234567890}], your next step can be)\n<respond> Joe Smith, 1234567890 </respond> OR <respond> The name is Joe Smith and the phone number is 1234567890 </respond>\n\n# Salesforce instance description\nThe objects available in the Salesforce instance are:\nUser, Account, Contact, Case, Knowledge__kav, ProductCategory, Product2, ProductCategoryProduct, Pricebook2, PricebookEntry, Order, OrderItem, EmailMessage, LiveChatTranscript, Issue__c, CaseHistory__c, Opportunity, OpportunityLineItem, Quote, QuoteLineItem, Contract, VoiceCallTranscript__c, Task, Event, Territory2, UserTerritory2Association, Lead\n\n## The fields available for the objects along with their descriptions and dependencies are:\nUser\n  - Alias: Alias, usually a shortened version of the agent's name\n  - Email: Email address of the agent\n  - EmailEncodingKey: Email encoding, usually 'UTF-8'\n  - FirstName: First name of the agent\n  - Id: ID of the user\n  - LanguageLocaleKey: Language locale, e.g., 'en_US'\n  - LastName: Last name of the agent\n  - LocaleSidKey: Locale, e.g., 'en_US'\n  - Phone: Phone number of the agent\n  - ProfileId: ID of the user profile, typically 'Standard User' or a similar profile\n  - TimeZoneSidKey: Time zone, e.g., 'America/Los_Angeles'\n  - Username: Unique username for the agent in Salesforce\nAccount\n  - Description: Detailed description of the account\n  - Id: ID of the account\n  - Industry: Industry the account belongs to\n  - Name: Name of the account\n  - NumberOfEmployees: Number of employees in the account\n  - Phone: Phone number of the account\n  - ShippingState: State for shipping address\nContact\n  - AccountId: References Account object\n  - Email: Email address of the contact\n  - FirstName: First name of the contact\n  - Id: ID of the contact\n  - LastName: Last name of the contact\nCase\n  - AccountId: References Account object\n  - ClosedDate: Date when the case was closed in 'YYYY-MM-DDTHH:MM:SSZ' format\n  - ContactId: ID of the contact associated with this case. References Contact object\n  - CreatedDate: Date when the case was created in 'YYYY-MM-DDTHH:MM:SSZ' format\n  - Description: Detailed description of the case\n  - Id: ID of the case\n  - IssueId__c: ID of the Issue associated with this case\n  - OrderItemId__c: ID of the Order Item associated with this case\n  - OwnerId: The latest owner of the case. References User object\n  - Priority: Priority of the case\n  - Status: Status of the case, e.g., 'New', 'Working', 'Escalated', etc\n  - Subject: Subject of the case\nKnowledge__kav\n  - FAQ_Answer__c: Content of the knowledge article\n  - Id: ID of the knowledge article\n  - Summary: Summary of the article\n  - Title: Title of the knowledge article\n  - UrlName: Unique URL for the article\nProductCategory\n  - CatalogId: ID of the catalog this category belongs to\n  - Id: ID of the product category\n  - Name: Name of the product category\nProduct2\n  - Description: Description of the product\n  - External_ID__c: External ID for mapping purposes\n  - Id: ID of the product\n  - IsActive: Boolean to mark if the product is active\n  - Name: Name of the product\nProductCategoryProduct\n  - Id: ID of the product category product\n  - ProductCategoryId: References ProductCategory object\n  - ProductId: References Product2 object\nPricebook2\n  - Description: Description of the price book\n  - Id: ID of the price book\n  - IsActive: Boolean to mark if the price book is active\n  - Name: Name of the price book\n  - ValidFrom: Start date for the price book in 'YYYY-MM-DDTHH:MM:SSZ' format\n  - ValidTo: End date for the price book in 'YYYY-MM-DDTHH:MM:SSZ' format\nPricebookEntry\n  - Id: ID of the price book entry\n  - Pricebook2Id: ID of the price book this entry belongs to. References Pricebook2 object\n  - Product2Id: ID of the product associated with this price book entry. References Product2 object\n  - UnitPrice: Unit price of the product\nOrder\n  - AccountId: ID of the customer account. References Account object\n  - EffectiveDate: Effective date of the order in 'YYYY-MM-DD' format\n  - Id: ID of the order\n  - OwnerId: ID of the agent who owns the order. References User object\n  - Pricebook2Id: ID of the price book this order belongs to. References Pricebook2 object\n  - Status: Status of the order, typically 'Draft' or 'Activated'\nOrderItem\n  - Id: ID of the order item\n  - OrderId: ID of the order this item belongs to. References Order object\n  - PriceBookEntryId: ID of the price book entry for this order item. References PricebookEntry object\n  - Product2Id: ID of the product associated with this order item. References Product2 object\n  - Quantity: Quantity of the product ordered\n  - UnitPrice: Unit price of the product\nEmailMessage\n  - FromAddress: Sender email address\n  - Id: ID of the email message\n  - MessageDate: Timestamp when the message was sent in 'YYYY-MM-DDTHH:MM:SSZ' format\n  - ParentId: ID of the case related to this email. References Case object\n  - RelatedToId: ID that links to objects like Opportunity and Case. Identical to ParentId when associated with a case. References Case and Opportunity objects.\n  - Subject: Subject of the email\n  - TextBody: Content of the email\n  - ToIds: List of recipient IDs\nLiveChatTranscript\n  - AccountId: ID of the customer account. References Account object\n  - Body: Content of the chat transcript\n  - CaseId: ID of the case related to this chat. References Case object\n  - ContactId: References Contact object\n  - EndTime: Timestamp when the chat ended\n  - Id: ID of the live chat transcript\n  - LiveChatVisitorId: ID of the chat visitor\n  - OwnerId: ID of the agent who owned the chat. References User object\nIssue__c\n  - Description__c: Detailed description of the issue\n  - Id: ID of the issue\n  - Name: The name/title of the issue\nCaseHistory__c\n  - CaseId__c: ID of the case this history entry is related to. References Case object\n  - CreatedDate: The date when this change was made, in 'YYYY-MM-DDTHH:MM:SSZ' format\n  - Field__c: Specifies the field or type of change. One of ['Case Creation', 'Owner Assignment', 'Case Closed']\n  - Id: ID of the case history\n  - NewValue__c: The new value after the change. References User object if the type of change is 'Owner Assignment' else null\n  - OldValue__c: The previous value before the change. References User object if the type of change is 'Owner Assignment' else null\nOpportunity\n  - AccountId: ID of the customer account. References Account object\n  - Amount: Monetary value of the opportunity in the default currency\n  - CloseDate: Expected close date of the opportunity\n  - ContactId: ID of the contact associated with this opportunity. References Contact object\n  - ContractID__c: ID of the contract associated with this opportunity. References Contract object\n  - CreatedDate: Date when the opportunity was created, in 'YYYY-MM-DDTHH:MM:SSZ' format\n  - Description: Detailed description of the opportunity\n  - Id: ID of the opportunity\n  - Name: Name of the opportunity\n  - OwnerId: ID of the agent who owns the opportunity. References User object\n  - Probability: Probability of winning the opportunity\n  - StageName: Stage of the opportunity. One of ['Qualification', 'Discovery', 'Quote', 'Negotiation', 'Closed']\nOpportunityLineItem\n  - Id: ID of the opportunity line item\n  - OpportunityId: ID of the opportunity associated with this line item. References Opportunity object\n  - PricebookEntryId: ID of the price book entry for this line item. References PricebookEntry object\n  - Product2Id: ID of the product associated with this line item. References Product2 object\n  - Quantity: Quantity of the product ordered\n  - TotalPrice: Total price of the product. Calculated as Quantity * UnitPrice * (1 - Discount/100)\nQuote\n  - AccountId: ID of the customer account. References Account object\n  - ContactId: ID of the contact associated with this quote. References Contact object\n  - CreatedDate: Date when the quote was created, in 'YYYY-MM-DDTHH:MM:SSZ' format\n  - Description: Detailed description of the quote\n  - ExpirationDate: Expiration date of the quote, in 'YYYY-MM-DDTHH:MM:SSZ' format\n  - Id: ID of the quote\n  - Name: Name of the quote\n  - OpportunityId: ID of the opportunity associated with this quote. References Opportunity object\n  - Status: Status of the quote. One of ['Draft', 'Needs Review', 'In Review', 'Approved', 'Rejected', 'Presented', 'Accepted', 'Denied']\nQuoteLineItem\n  - Discount: Discount applied to the product\n  - Id: ID of the quote line item\n  - OpportunityLineItemId: ID of the opportunity line item associated with this quote line item. References OpportunityLineItem object\n  - PricebookEntryId: ID of the price book entry for this line item. References PricebookEntry object\n  - Product2Id: ID of the product associated with this line item. References Product2 object\n  - Quantity: Quantity of the product ordered\n  - QuoteId: ID of the quote associated with this line item. References Quote object\n  - TotalPrice: Total price of the product. Calculated as Quantity * UnitPrice * (1 - Discount/100)\n  - UnitPrice: Unit price of the product\nContract\n  - AccountId: ID of the customer account. References Account object\n  - CompanySignedDate: Date when the company signed the contract, in 'YYYY-MM-DD' format\n  - ContractTerm: Duration of the contract in months\n  - CustomerSignedDate: Date when the customer signed the contract, in 'YYYY-MM-DD' format\n  - Description: Detailed description of the contract terms and conditions\n  - Id: ID of the contract\n  - StartDate: Date when the contract becomes effective, in 'YYYY-MM-DD' format\n  - Status: Status of the contract. One of ['In approval Process', 'Activated', 'Draft', 'Rejected', 'Negotiating', 'Awaiting Signature', 'Signature Declined', 'Signed', 'Canceled', 'Contract Terminated']\nVoiceCallTranscript__c\n  - Body__c: Content of the voice call transcript\n  - CreatedDate: Date when the voice call transcript was created, in 'YYYY-MM-DDTHH:MM:SSZ' format\n  - EndTime__c: Timestamp when the voice call transcript ended, in 'YYYY-MM-DDTHH:MM:SSZ' format\n  - Id: ID of the voice call transcript\n  - LeadId__c: ID of the lead associated with this voice call transcript. References Lead object\n  - OpportunityId__c: ID of the opportunity associated with this voice call transcript. References Opportunity object\nTask\n  - ActivityDate: Date when the task is due, in 'YYYY-MM-DDTHH:MM:SSZ' format\n  - Description: Detailed description of the task\n  - Id: ID of the task\n  - OwnerId: ID of the agent who owns the task. References User object\n  - Priority: Priority of the task. One of ['Low', 'Medium', 'High', 'Urgent']\n  - Status: Status of the task. One of ['Not Started', 'In Progress', 'Completed', 'Deferred', 'Waiting']\n  - Subject: Subject of the task\n  - WhatId: ID of the opportunity associated with this task. References Opportunity object\nEvent\n  - Description: Detailed description of the event\n  - DurationInMinutes: Duration of the event in minutes\n  - Id: ID of the event\n  - IsAllDayEvent: Boolean to mark if the event is an all-day event\n  - Location: Location of the event\n  - OwnerId: ID of the agent who owns the event. References User object\n  - StartDateTime: Start datetime of the event, in 'YYYY-MM-DDTHH:MM:SSZ' format\n  - Subject: Subject of the event\n  - WhatId: ID of the opportunity associated with this event. References Opportunity object\nTerritory2\n  - Description: A comma separated list of territories/states (e.g. CA,IL,AL)\n  - Id: ID of the territory\n  - Name: Name of the territory\nUserTerritory2Association\n  - Id: ID of the user territory association\n  - Territory2Id: ID of the territory associated with the agent. References Territory2 object\n  - UserId: ID of the agent who is associated with the territory. References User object\nLead\n  - Company: Company name of the lead\n  - ConvertedAccountId: ID of the Account record created when the Lead is converted. References Account object\n  - ConvertedContactId: ID of the Contact record created when the Lead is converted. References Contact object\n  - ConvertedDate: Date when the lead was converted in 'YYYY-MM-DD' format\n  - CreatedDate: Date when the lead was created in 'YYYY-MM-DDTHH:MM:SSZ' format\n  - Email: Email address of the lead\n  - FirstName: First name of the lead\n  - Id: ID of the lead\n  - IsConverted: Boolean indicating whether the lead has been converted\n  - LastName: Last name of the lead\n  - OwnerId: ID of the agent who owns the lead. References User object\n  - Phone: Phone number of the lead\n  - Status: Status of the lead. One of ['New', 'Contacted', 'Qualified', 'Proposal', 'Negotiation', 'Closed']\n  - Title: Job title of the lead\n\n# Salesforce instance Metadata\n\n        ## Lead qualification guide.\n        Look for the voice call transcripts with the lead and relevant knowledge articles to justify the lead qualification.\n        \n        - Lead Id to be considered is: 00QWt0000089AekMAE"},
  {'role': 'user',
   'content': "Can this lead be qualified based on the latest discussions? If the answer is no, which factors—'Budget', 'Authority', 'Need', or 'Timeline'—are responsible? Return only one or several of the four BANT factors that the lead qualification fails to meet (i.e. 'Budget', 'Authority', 'Need', 'Timeline')."}]

In [17]:
MODEL = "bedrock/amazon.nova-lite-v1:0"

In [18]:
hello_resp = completion(
    # api_key=gcp_api,
    model=MODEL,
    messages=[{"role":"user","content":"Hello!"}],
    # messages=test_q,
)

In [21]:
hello_resp

ModelResponse(id='chatcmpl-1e9678f8-6f58-4a0c-b8b1-bd125de695bb', created=1763570599, model='amazon.nova-lite-v1:0', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello! How can I assist you today? If you have any questions or need information on a particular topic, feel free to ask. Whether it's about science, technology, history, or something else, I'm here to help.", role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=49, prompt_tokens=2, total_tokens=51, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None, cache_creation_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0))

In [98]:


multiple_responses = []
for i in range(3):
    resp = completion(
        model=MODEL,
        # messages=[{"role":"user","content":"Hello!"}],
        messages=test_q,
        temperature=i/10
    )
    multiple_responses.append(resp.choices[0].message["content"])

In [99]:
multiple_responses

["<execute> SELECT Id, VoiceCallTranscript__c.Body__c, VoiceCallTranscript__c.CreatedDate, VoiceCallTranscript__c.LeadId__c, Knowledge__kav.Title, Knowledge__kav.Summary FROM VoiceCallTranscript__c WHERE LeadId__c = '00QWt0000089AekMAE' LIMIT 1 </execute>",
 "<execute> SELECT Id, VoiceCallTranscript__c.Body__c, VoiceCallTranscript__c.CreatedDate, Knowledge__kav.Summary, Knowledge__kav.Title FROM Lead WHERE Id = '00QWt0000089AekMAE' AND IsConverted = false </execute>",
 "<execute> SELECT Id, VoiceCallTranscript__c.Body__c, VoiceCallTranscript__c.CreatedDate, Knowledge__kav.Summary, Knowledge__kav.Title FROM Lead WHERE Id = '00QWt0000089AekMAE' AND IsConverted = false </execute>"]

In [26]:
import boto3, json
region = "us-east-1"
model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"  # exact ID, no 'us.'
brt = boto3.client("bedrock-runtime", region_name=region)

# 1) Converse (should work if allowed)
try:
    out = brt.converse(
        modelId=model_id,
        messages=[{"role":"user","content":[{"text":"hello"}]}],
        inferenceConfig={"temperature":0.1}
    )
    print("Converse OK:", out["output"]["message"]["content"][0]["text"])
except Exception as e:
    print("Converse ERR:", e)

# 2) Invoke fallback (often works when Converse is gated)
try:
    body = {"anthropic_version":"bedrock-2023-05-31","max_tokens":64,"messages":[{"role":"user","content":"hello"}]}
    out = brt.invoke_model(modelId=model_id, body=json.dumps(body))
    print("Invoke OK:", json.loads(out["body"].read()))
except Exception as e:
    print("Invoke ERR:", e)


Converse ERR: An error occurred (ValidationException) when calling the Converse operation: Operation not allowed
Invoke ERR: An error occurred (ValidationException) when calling the InvokeModel operation: Operation not allowed
